In [2]:
import mysql.connector

# Connect to MySQL
conn = mysql.connector.connect(
    host="localhost",
    user="root",
    password="root",
    auth_plugin="mysql_native_password"

)

cursor = conn.cursor()

In [5]:

cursor.execute("CREATE DATABASE IF NOT EXISTS college_db")
cursor.execute("USE college_db")


cursor.execute("DROP VIEW IF EXISTS student_totals")
cursor.execute("DROP TABLE IF EXISTS student_log")
cursor.execute("DROP TABLE IF EXISTS marks")
cursor.execute("DROP TABLE IF EXISTS course_enrollments")
cursor.execute("DROP TABLE IF EXISTS courses")
cursor.execute("DROP TABLE IF EXISTS student_info")

# Create student_info table
cursor.execute("""
CREATE TABLE student_info (
    id INT PRIMARY KEY,
    name VARCHAR(50),
    age INT,
    department VARCHAR(50),
    email VARCHAR(100)
)
""")

# Insert students
cursor.executemany("""
INSERT INTO student_info (id, name, age, department, email)
VALUES (%s, %s, %s, %s, %s)
""", [
    (1, 'Alice', 19, 'Computer Science', 'alice@example.com'),
    (2, 'Bob', 22, 'Physics', 'bob@example.com'),
    (3, 'Charlie', 20, 'Mathematics', 'charlie@example.com'),
    (4, 'John', 21, 'History', 'john@example.com'),
    (5, 'Daisy', 23, 'Computer Science', 'daisy@example.com')
])
conn.commit()

# Create courses table
cursor.execute("""
CREATE TABLE courses (
    course_id INT PRIMARY KEY,
    course_name VARCHAR(100),
    credits INT
)
""")

cursor.executemany("""
INSERT INTO courses (course_id, course_name, credits)
VALUES (%s, %s, %s)
""", [
    (101, 'Database Systems', 4),
    (102, 'Operating Systems', 3),
    (103, 'Data Structures', 4)
])
conn.commit()

# Create marks table
cursor.execute("DROP TABLE IF EXISTS marks")
cursor.execute("""
CREATE TABLE marks (
    student_id INT,
    subject VARCHAR(50),
    marks INT,
    FOREIGN KEY (student_id) REFERENCES student_info(id)
)
""")

cursor.executemany("""
INSERT INTO marks (student_id, subject, marks)
VALUES (%s, %s, %s)
""", [
    (1, 'Math', 85),
    (1, 'Science', 90),
    (2, 'Math', 78),
    (2, 'Science', 88),
    (4, 'History', 95)
])
conn.commit()

# JOIN students and marks
cursor.execute("""
SELECT s.name, m.subject, m.marks
FROM student_info s
JOIN marks m ON s.id = m.student_id
""")
print("\nJoined Data:", cursor.fetchall())

# 29. Average marks
cursor.execute("""
SELECT student_id, AVG(marks) as avg_marks
FROM marks
GROUP BY student_id
""")
print("\nAverage Marks:", cursor.fetchall())

# 30. Total marks per student
cursor.execute("""
SELECT student_id, SUM(marks) as total_marks
FROM marks
GROUP BY student_id
""")
print("\nTotal Marks:", cursor.fetchall())

# 31. Students with total > 200
cursor.execute("""
SELECT student_id, SUM(marks) as total
FROM marks
GROUP BY student_id
HAVING total > 200
""")
print("\nMarks > 200:", cursor.fetchall())

# 32. Students with same age
cursor.execute("""
SELECT age, COUNT(*)
FROM student_info
GROUP BY age
HAVING COUNT(*) > 1
""")
print("\nStudents with same age:", cursor.fetchall())

# 33. JOIN types
cursor.execute("""
SELECT s.name, m.subject
FROM student_info s
INNER JOIN marks m ON s.id = m.student_id
""")
print("\nInner Join:", cursor.fetchall())

cursor.execute("""
SELECT s.name, m.subject
FROM student_info s
LEFT JOIN marks m ON s.id = m.student_id
""")
print("\nLeft Join:", cursor.fetchall())

cursor.execute("""
SELECT s.name, m.subject
FROM marks m
RIGHT JOIN student_info s ON s.id = m.student_id
""")
print("\nRight Join:", cursor.fetchall())

# 34. Auto increment table
cursor.execute("""
CREATE TABLE IF NOT EXISTS auto_table (
    auto_id INT AUTO_INCREMENT PRIMARY KEY,
    data VARCHAR(50)
)
""")

# 35. Foreign key referencing another table
cursor.execute("""
CREATE TABLE IF NOT EXISTS course_enrollments (
    enrollment_id INT AUTO_INCREMENT PRIMARY KEY,
    student_id INT,
    course_id INT,
    FOREIGN KEY (student_id) REFERENCES student_info(id),
    FOREIGN KEY (course_id) REFERENCES courses(course_id)
)
""")

# 36. Max marks
cursor.execute("SELECT MAX(marks) FROM marks")
print("\nMax Marks:", cursor.fetchone())

# 37. View
cursor.execute("""
CREATE OR REPLACE VIEW student_totals AS
SELECT s.name, SUM(m.marks) as total
FROM student_info s
JOIN marks m ON s.id = m.student_id
GROUP BY s.name
""")
cursor.execute("SELECT * FROM student_totals")
print("\nView student_totals:", cursor.fetchall())

# 38. Subquery greater than avg
cursor.execute("""
SELECT name
FROM student_info
WHERE id IN (
    SELECT student_id FROM marks
    GROUP BY student_id
    HAVING AVG(marks) > (
        SELECT AVG(marks) FROM marks
    )
)
""")
print("\nStudents > Avg Marks:", cursor.fetchall())

# 39. Stored procedure
cursor.execute("DROP PROCEDURE IF EXISTS insert_student")
cursor.execute("""
CREATE PROCEDURE insert_student (
    IN sid INT,
    IN sname VARCHAR(50),
    IN sage INT,
    IN sdept VARCHAR(50),
    IN semail VARCHAR(100)
)
BEGIN
    INSERT INTO student_info (id, name, age, department, email)
    VALUES (sid, sname, sage, sdept, semail);
END;
""", multi=True)

# 40. Procedure to update department
cursor.execute("DROP PROCEDURE IF EXISTS update_department")
cursor.execute("""
CREATE PROCEDURE update_department(
    IN sid INT,
    IN new_dept VARCHAR(50)
)
BEGIN
    UPDATE student_info SET department = new_dept WHERE id = sid;
END;
""", multi=True)

# 41. User-defined function
cursor.execute("DROP FUNCTION IF EXISTS get_grade")
cursor.execute("""
CREATE FUNCTION get_grade(marks INT) RETURNS CHAR(1)
DETERMINISTIC
BEGIN
    DECLARE grade CHAR(1);
    IF marks >= 90 THEN SET grade = 'A';
    ELSEIF marks >= 75 THEN SET grade = 'B';
    ELSE SET grade = 'C';
    END IF;
    RETURN grade;
END;
""", multi=True)

# 42. Trigger
cursor.execute("DROP TRIGGER IF EXISTS after_student_insert")
cursor.execute("DROP TABLE IF EXISTS student_log")
cursor.execute("""
CREATE TABLE student_log (
    log_id INT AUTO_INCREMENT PRIMARY KEY,
    student_id INT,
    action_time TIMESTAMP DEFAULT CURRENT_TIMESTAMP
)
""")
cursor.execute("""
CREATE TRIGGER after_student_insert
AFTER INSERT ON student_info
FOR EACH ROW
BEGIN
    INSERT INTO student_log(student_id) VALUES(NEW.id);
END;
""", multi=True)

# 43. Transaction
try:
    conn.start_transaction()
    cursor.execute("UPDATE student_info SET age = age + 1 WHERE id = 1")
    cursor.execute("UPDATE student_info SET age = age + 1 WHERE id = 2")
    conn.commit()
    print("\nTransaction committed successfully.")
except:
    conn.rollback()
    print("\nTransaction rolled back.")

# 44. Duplicate records
cursor.execute("""
SELECT name, COUNT(*)
FROM student_info
GROUP BY name
HAVING COUNT(*) > 1
""")
print("\nDuplicate Records:", cursor.fetchall())

# 45. Backup guidance
print("\nBackup using: mysqldump -u root -p college_db > backup.sql")

# 46. Restore guidance
print("\nRestore using: mysql -u root -p college_db < backup.sql")

# 47. CSV import guidance
print("\nExample: LOAD DATA INFILE 'file.csv' INTO TABLE student_info FIELDS TERMINATED BY ',' LINES TERMINATED BY '\n' (id, name, age, department, email);")

# 48. Index creation
cursor.execute("CREATE INDEX idx_name ON student_info(name)")
print("\nIndex created on student_info(name)")

# 49. Second highest mark
cursor.execute("""
SELECT MAX(marks) FROM marks
WHERE marks < (SELECT MAX(marks) FROM marks)
""")
print("\nSecond Highest Mark:", cursor.fetchone())

# 50. Drop courses table
cursor.execute("DROP TABLE IF EXISTS course_enrollments")
cursor.execute("DROP TABLE IF EXISTS courses")
print("\nCourses table dropped.")



Joined Data: [('Alice', 'Math', 85), ('Alice', 'Science', 90), ('Bob', 'Math', 78), ('Bob', 'Science', 88), ('John', 'History', 95)]

Average Marks: [(1, Decimal('87.5000')), (2, Decimal('83.0000')), (4, Decimal('95.0000'))]

Total Marks: [(1, Decimal('175')), (2, Decimal('166')), (4, Decimal('95'))]

Marks > 200: []

Students with same age: []

Inner Join: [('Alice', 'Math'), ('Alice', 'Science'), ('Bob', 'Math'), ('Bob', 'Science'), ('John', 'History')]

Left Join: [('Alice', 'Math'), ('Alice', 'Science'), ('Bob', 'Math'), ('Bob', 'Science'), ('Charlie', None), ('John', 'History'), ('Daisy', None)]

Right Join: [('Alice', 'Math'), ('Alice', 'Science'), ('Bob', 'Math'), ('Bob', 'Science'), ('Charlie', None), ('John', 'History'), ('Daisy', None)]

Max Marks: (95,)

View student_totals: [('Alice', Decimal('175')), ('Bob', Decimal('166')), ('John', Decimal('95'))]

Students > Avg Marks: [('Alice',), ('John',)]

Transaction committed successfully.

Duplicate Records: []

Backup using: my